Welcome to the Smart SQL Chatbot project! In this project, I built a chatbot that can talk to a SQLite database. It uses Google GenAI to automatically turn plain English questions into SQL queries, which are then executed against a sample computer store database. I built the database using SQLite and even set up some helper functions (like `list_tables`, `describe_table`, and `execute_query`). The whole idea is to create a fun, interactive tool that bridges natural language and SQL queries.



## Overview

This project is designed to be run in a Jupyter Notebook. It:
- **Sets up a sample SQLite database** with tables for products, staff, and orders.
- **Defines helper functions** to list tables, describe table schemas, and execute select queries.
- **Initializes the Google GenAI chat client** with a custom system instruction so that the chatbot uses these functions as tools.
- **Listens to user queries** and automatically converts them into SQL queries to retrieve the needed information.

## Prerequisites

Before you get started, make sure you have:
- Python installed (preferably in an environment like Jupyter Notebook).
- The following Python packages:
  - `google-genai==1.7.0`
  - `ipython-sql`
  - `sqlite3` (which comes with Python)
- A valid `GOOGLE_API_KEY` stored in your Kaggle Secrets (or your preferred secret management).

## Setup Instructions

1. **Uninstall Conflicting Packages and Install Dependencies**

   Run the following commands in a notebook cell:
   ```python
   !pip uninstall -qqy jupyterlab  # Remove unused conflicting packages
   !pip install -U -q "google-genai==1.7.0"


Import Libraries and Set Up the GenAI Client

In the next cell, import the necessary modules, retrieve your API key, and set up a retry policy to handle quota limits:

In [ ]:
!pip uninstall -qqy jupyterlab  # Remove unused conflicting packages
!pip install -U -q "google-genai==1.7.0"
import sqlite3
from google import genai
from google.genai import types
print("google-genai version:", genai.__version__)

from kaggle_secrets import UserSecretsClient
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

# Define a retry policy. This ensures that if the model makes several consecutive calls (e.g. for a complex query)
# it will retry automatically upon hitting quota limits (HTTP 429 or 503).
from google.api_core import retry
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})
if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
    genai.models.Models.generate_content = retry.Retry(
        predicate=is_retriable)
    (genai.models.Models.generate_content)

Load SQL Extension and Create the Database

Use the SQL magic in a cell to create and populate your SQLite database:

In [ ]:
%load_ext sql
%sql sqlite:///sample.db

%%sq
DROP TABLE IF EXISTS orders;
DROP TABLE IF EXISTS staff;
DROP TABLE IF EXISTS products;

CREATE TABLE IF NOT EXISTS products (
    product_id INTEGER PRIMARY KEY AUTOINCREMENT,
    product_name VARCHAR(255) NOT NULL,
    product_description TEXT,
    category VARCHAR(100),
    price DECIMAL(10, 2) NOT NULL,
    stock_quantity INTEGER DEFAULT 0,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

CREATE TABLE IF NOT EXISTS staff (
    staff_id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name VARCHAR(255) NOT NULL,
    last_name VARCHAR(255) NOT NULL,
    email VARCHAR(255),
    department VARCHAR(100),
    hire_date DATE DEFAULT CURRENT_DATE
);

CREATE TABLE IF NOT EXISTS orders (
    order_id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_name VARCHAR(255) NOT NULL,
    staff_id INTEGER NOT NULL,
    product_id INTEGER NOT NULL,
    order_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    shipping_address VARCHAR(255),
    order_status VARCHAR(50) DEFAULT 'Pending',
    FOREIGN KEY (staff_id) REFERENCES staff (staff_id),
    FOREIGN KEY (product_id) REFERENCES products (product_id)
);

INSERT INTO products (product_name, product_description, category, price, stock_quantity) VALUES
   ('Laptop', 'High performance laptop with 16GB RAM and 512GB SSD for demanding tasks.', 'Electronics', 799.99, 50),
   ('Keyboard', 'Ergonomic mechanical keyboard with customizable backlighting.', 'Electronics', 129.99, 150),
   ('Mouse', 'Wireless optical mouse with advanced tracking technology.', 'Electronics', 29.99, 300),
   ('Smartphone', 'Latest smartphone featuring an edge-to-edge display and advanced camera setup.', 'Electronics', 999.99, 100),
   ('Desk Chair', 'Ergonomic office chair with adjustable lumbar support and breathable mesh.', 'Furniture', 199.99, 75);

INSERT INTO staff (first_name, last_name, email, department) VALUES
   ('Alice', 'Smith', 'alice.smith@example.com', 'Sales'),
   ('Bob', 'Johnson', 'bob.johnson@example.com', 'Customer Support'),
   ('Charlie', 'Williams', 'charlie.williams@example.com', 'Logistics'),
   ('Diana', 'Brown', 'diana.brown@example.com', 'Management');

INSERT INTO orders (customer_name, staff_id, product_id, shipping_address, order_status) VALUES
   ('David Lee', 1, 1, '123 Main St, Anytown, USA', 'Shipped'),
   ('Emily Chen', 2, 2, '456 Oak Ave, Somewhere, USA', 'Processing'),
   ('Frank Brown', 1, 3, '789 Pine Rd, Elsewhere, USA', 'Delivered'),
   ('Grace Hopper', 3, 4, '101 Maple St, Treecity, USA', 'Pending'),
   ('Hannah Wilson', 2, 5, '202 Birch Blvd, Woodville, USA', 'Cancelled');


Define Helper Functions for Database Interaction

Next, set up your Python functions to list tables, describe a table, and execute queries:

In [ ]:
# Connect to the SQLite database using the sqlite3 module.
db_file = "sample.db"
db_conn = sqlite3.connect(db_file)

def list_tables() -> list[str]:
    """Retrieve the names of all tables in the database."""
    print(" - DB CALL: list_tables()")
    cursor = db_conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    return [t[0] for t in tables]

print("Tables:", list_tables())

def describe_table(table_name: str) -> list[tuple[str, str]]:
    """Return a list of (column name, type) tuples for the given table."""
    print(f" - DB CALL: describe_table({table_name})")
    cursor = db_conn.cursor()
    cursor.execute(f"PRAGMA table_info({table_name});")
    schema = cursor.fetchall()
    return [(col[1], col[2]) for col in schema]

print("Schema for products:", describe_table("products"))

def execute_query(query: str) -> str:
    """Execute a SELECT SQL query and return the results as a string."""
    print(f" - DB CALL: execute_query({query})")
    cursor = db_conn.cursor()
    cursor.execute(query)
    results = cursor.fetchall()
    return str(results)

print("Query Products:", execute_query("SELECT * FROM products;"))


Initialize the Chatbot and Interact with the Database

Finally, set up the GenAI chat session by passing the database functions (tools) and a system instruction:

In [ ]:
# Define the list of Python functions that will be used as tools by the chatbot.
db_tools = [list_tables, describe_table, execute_query]

instruction = """You are a helpful chatbot that can interact with an SQL database
for a computer store. You will take the user's questions and turn them into SQL
queries using the tools available. Once you have the information you need, you will
answer the user's question using the data returned.

Use list_tables to see which tables are present, describe_table to understand the
schema, and execute_query to issue SQL SELECT queries.
"""

# Initialize the GenAI client using our Google API key.
client = genai.Client(api_key=GOOGLE_API_KEY)

# Start a chat session with automatic function calling enabled.
chat = client.chats.create(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=instruction,
        tools=db_tools,
    ),
)

# Send a test message to the chat and print the direct response.
response1 = chat.send_message("What is the cheapest product?")
print("Chat Response 1:", response1.text)

# Send another test message (this may build on the previous conversation).
response2 = chat.send_message("What products should salesperson Alice focus on to round out her portfolio? Explain why.")
print("Chat Response 2:", response2.text)


Initialize the Chatbot and Interact with the Database

Finally, set up the GenAI chat session by passing the database functions (tools) and a system instruction:

In [ ]:
# Define the list of Python functions that will be used as tools by the chatbot.
db_tools = [list_tables, describe_table, execute_query]

instruction = """You are a helpful chatbot that can interact with an SQL database
for a computer store. You will take the user's questions and turn them into SQL
queries using the tools available. Once you have the information you need, you will
answer the user's question using the data returned.

Use list_tables to see which tables are present, describe_table to understand the
schema, and execute_query to issue SQL SELECT queries.
"""

# Initialize the GenAI client using our Google API key.
client = genai.Client(api_key=GOOGLE_API_KEY)

# Start a chat session with automatic function calling enabled.
chat = client.chats.create(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=instruction,
        tools=db_tools,
    ),
)

# Send a test message to the chat and print the direct response.
response1 = chat.send_message("What is the cheapest product?")
print("Chat Response 1:", response1.text)

# Send another test message (this may build on the previous conversation).
response2 = chat.send_message("What products should salesperson Alice focus on to round out her portfolio? Explain why.")
print("Chat Response 2:", response2.text)


(Optional) View Detailed Chat History

If you want to inspect the chat history or see the function calls:

In [ ]:
import textwrap

def print_chat_turns(chat_instance):
    """Print the chat history including any function calls and responses."""
    for event in chat_instance.get_history():
        print(f"{event.role.capitalize()}:")
        for part in event.parts:
            if part.text:
                print(f"  \"{part.text}\"")
            elif part.function_call:
                args = ", ".join(f"{k}={v}" for k, v in part.function_call.args.items())
                print(f"  Function call: {part.function_call.name}({args})")
            elif part.function_response:
                print("  Function response:")
                print(textwrap.indent(str(part.function_response.response['result']), "    "))
        print()

print_chat_turns(chat)

# Optional asynchronous function to handle streaming responses.
import asyncio
from pprint import pformat
from IPython.display import display, Image, Markdown

async def handle_response(stream, tool_impl=None):
  """Stream output and handle any tool calls during the session."""
  all_responses = []
  async for msg in stream.receive():
    all_responses.append(msg)
    if msg.text:
      display(Markdown("### Text"))
      print(msg.text, end='')
    elif msg.tool_call:
      display(Markdown("### Tool call"))
      for fc in msg.tool_call.function_calls:
        try:
          result = tool_impl(**fc.args) if callable(tool_impl) else "ok"
        except Exception as e:
          result = str(e)
        tool_response = types.LiveClientToolResponse(
            function_responses=[types.FunctionResponse(
                name=fc.name,
                id=fc.id,
                response={'result': result},
            )]
        )
        await stream.send(input=tool_response)
    elif msg.server_content and msg.server_content.model_turn:
      for part in msg.server_content.model_turn.parts:
          if part.executable_code:
            display(Markdown(f'### Code\n```\n{part.executable_code.code}\n```'))
          elif part.code_execution_result:
            display(Markdown(f'### Result: {part.code_execution_result.outcome}\n```\n{pformat(part.code_execution_result.output)}\n```'))
          elif part.inline_data:
            display(Image(part.inline_data.data))
  return all_responses

# (You could later call: await handle_response(your_stream, tool_impl=execute_query))


In [ ]:
├── README.md         # This file
├── Notebook.ipynb    # Jupyter Notebook with all the code cells described above
└── sample.db         # SQLite database file (created when you run the notebook)